In [17]:
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import kagglehub
import torch
import os
%matplotlib inline

In [18]:
def download_dataset() -> str:
    path = kagglehub.dataset_download("rishitjakharia/names-txt")
    return path # for easier access later

In [19]:
dataset_path = download_dataset() + "/names.txt"

In [20]:
words = open(dataset_path, 'r').read().splitlines()

In [21]:
def make_int_char_maps() -> tuple[dict, dict]:
  chars = sorted(list(set(''.join(words))))
  ctoi = {c: i + 1 for i, c in enumerate(chars)}
  ctoi['.'] = 0
  itoc = {i: c for c, i in ctoi.items()}
  return ctoi, itoc

In [22]:
ctoi, itoc = make_int_char_maps()

In [23]:
print(itoc)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [24]:
print(ctoi)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}


In [25]:
vocab_size = len(ctoi)
print(vocab_size)

27


In [26]:
def build_dataset(inp_words) -> tuple[torch.tensor, torch.tensor]:

  block_size = 3 # context length -> how many chars does it take to predict the next
  inp, target = [], []
  for w in inp_words:
    context = [0] * block_size

    for ch in w + '.':
      ix = ctoi[ch]
      inp.append(context)
      target.append(ix)
      context = context[1:] + [ix] # crop and append

  inp = torch.tensor(inp)
  target = torch.tensor(target)
  return inp, target

In [27]:
import random
random.seed(42)

random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [28]:
# to later compare gradients we computed to Pytorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [29]:
n_emb = 10
n_context = 3
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_emb), generator=g)

# first layer                                           # kaiming init
W1 = torch.randn((n_emb * n_context, n_hidden), generator=g) * (5/3)/((n_emb * 3) ** 0.5)
B1 = torch.randn(n_hidden, generator=g) * 0.1 # these biases dont do anything since we saw they get subtracted in batch normlizing but its here for fun

# second layer
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
B2 = torch.randn(vocab_size, generator=g) * 0.1

# batchnorm params
bngain = torch.ones((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.zeros((1, n_hidden)) * 0.1

parameters = [C, W1, B1, W2, B2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True




4137


In [30]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [47]:
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)

# linear layer 1
hprebn = embcat @ W1 + B1
# batch norm
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# non linearity
h = torch.tanh(hpreact)
# second linear layer
logits = h @ W2 + B2
# cross entropy loss
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract the max so we dont get inf in counts
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum ** -1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(Xb.shape[0]), Yb].mean()

# pytorch backwards
for p in parameters:
  p.grad = None

for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
          embcat, emb, bndiff2]: # Added bndiff2
  t.retain_grad()

loss.backward()
loss

tensor(3.3482, grad_fn=<NegBackward0>)

In [49]:
from ctypes import c_size_t
# Ecercise 1
# Calculating all of the deriatives manually to do backprop

# L = loss
# dL/dL = 1

# dL/dlogprobs
# loss = -(a + b + c ... ) /B
#      = -a/B -b/B -c/B ...
#      = dL/da = -1/B, dL/db = -1/B ...
# adding just propates the gradient
# dl/dlogprobs = -1/B

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n

# dL/dprobs = dL/dlogprobs * dlogprobs/dprobs -> chain rule
#           = -1/B         * ???
# logprobs = probs.log()
# logprobs = ln(probs)
# dlogprobs/dprobs = 1/probs

# dL/dprobs = dlogprobs * 1/probs
dprobs = dlogprobs * 1.0/probs

# probs = counts * counts_sum_inv

# p  = a * b
# a[3x3] * b[3x1]
# a11*b1 a12*b1 a13*b1
# a21*b2 a22*b2 a23*b2
# a31*b3 a23*b3 a33*b3
# p[3x3]

# since we reuse b1, b2, b3 we need to add their gradients and keep the original shape [32x1]
# dL/dcsi = (counts * dprobs).sum(1, keepdim=true)
dcsi= (counts * dprobs).sum(1, keepdim=True)

# refering to our p = a * b
# a represents counts
# dL/da = dL/dp * dp/da
#       = dprobs * dcsi
dcounts = dprobs * counts_sum_inv

# counts_sum_inv = counts_sum ** -1
# dcounts_sum = dL/dcsi * -1 * counts_sum**-2
dcounts_sum = dcsi * (-counts_sum**-2)

# counts_sum = counts.sum(1, keepdims=True)
# addition pretty much carries the gradient over
# counts_sum = c1 + c2 + c3 ...
# dL/dc1 = dL/dcounts_sum * dcounts_sum/dc1
#        = dcounts_sum * 1
# we want to add to the already existing dcounts since its apart of multiple nodes in the prop tree

dcounts += torch.ones_like(counts) * dcounts_sum

#counts = norm_logits.exp()
#counts = e^norm_logits
# dL/dnorm_logits = dL/dcounts * dcounts/dnorm_logits
#                 = dcounts    * e^norm_logits

dnorm_logits = counts * dcounts


#norm_logits = logits - logit_maxes
# similar to out counts and counts_sum situtation, we repeat logit_maxes
# c  = a - b
# a[3x3] - b[3x1]
# a11-b1 a12-b1 a13-b1
# a21-b2 a22-b2 a23-b2
# a31-b3 a23-b3 a33-b3
# c[3x3]
#dL/dlogit_maxes = dnorm_logits * dnorm_logits/dlogit_maxes
#                               * [-1, -1, -1, ...]
#                                 [-1, -1, -1, ...]
#                                        ...

# dL/dlogits = dnorm_logits
dlogits = dnorm_logits.clone()
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)

# we arent done yet with dlogits though
# logit_maxes = logits.max(1, keepdim=True).values
# dL/logits = dlogit_maxes * dlogit_maxes/dlogits

#             dlogit_maxes/dlogits = 1 at max index per row. Everywhere else = 0

#            = dlogit_maxes * 1, for the max index in each row

#           dlogit_maxes linearly depends on dlogits since all the values in dlogit_maxes are in dlogits
# only the max value participates in the deriative
# since we are adding dlogits to
# dlogit_maxes = [b1]
#                [b2]
#                 ..
#                [b32]
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes


# I decided to work on the deratives on paper rather than commenting them
# don't want to spend too much time commenting each and every operation for visualization :)

dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)
dhpreact = (1.0 - h**2) * dh
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0, keepdim=True)
dbndiff = bnvar_inv * dbnraw
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
dbndiff += (2*bndiff) * dbndiff2
dhprebn = dbndiff.clone()
dbnmeani = (-dbndiff).sum(0)
dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
demb = dembcat.view(emb.shape)
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
  for j in range(Xb.shape[1]):
    ix = Xb[k,j]
    dC[ix] += demb[k,j]


cmp('dlogprobs', dlogprobs, logprobs)
cmp('dprobs', dprobs, probs)
cmp('dcsi', dcsi, counts_sum_inv)
cmp('dcounts_sum', dcounts_sum, counts_sum)
cmp('dcounts', dcounts, counts)
cmp('dnorm_logits', dnorm_logits, norm_logits)
cmp('dlogit_maxes', dlogit_maxes, logit_maxes)
cmp('dlogit', dlogits, logits)
cmp('dh', dh, h)
cmp('dw2', dW2, W2)
cmp('db2', db2, B2)
cmp('dhpreact', dhpreact, hpreact)
cmp('dbngain', dbngain, bngain)
cmp('dbnraw', dbnraw, bnraw)
cmp('dbnbias', dbnbias, bnbias)
cmp('dbnvar_inv', dbnvar_inv, bnvar_inv)
cmp('ddbnvar', dbnvar_inv, bnvar_inv)
cmp('dbndiff2', dbndiff2, bndiff2)
cmp('dbndiff', dbndiff, bndiff)
cmp('dbnmeani', dbnmeani, bnmeani)
cmp('dhprebn', dhprebn, hprebn)
cmp('dembcat', dembcat, embcat)
cmp('dW1', dW1, W1)
cmp('db1', db1, B1)
cmp('demb', demb, emb)
cmp('dC', dC, C)


# even thought approximations starts to become False as we go down,
# the calculations are correct -> refering to andrej karpathy lecture
# seems like running his jupyter notebook looked like the same thing


dlogprobs       | exact: True  | approximate: True  | maxdiff: 0.0
dprobs          | exact: True  | approximate: True  | maxdiff: 0.0
dcsi            | exact: True  | approximate: True  | maxdiff: 0.0
dcounts_sum     | exact: True  | approximate: True  | maxdiff: 0.0
dcounts         | exact: True  | approximate: True  | maxdiff: 0.0
dnorm_logits    | exact: True  | approximate: True  | maxdiff: 0.0
dlogit_maxes    | exact: True  | approximate: True  | maxdiff: 0.0
dlogit          | exact: True  | approximate: True  | maxdiff: 0.0
dh              | exact: True  | approximate: True  | maxdiff: 0.0
dw2             | exact: True  | approximate: True  | maxdiff: 0.0
db2             | exact: True  | approximate: True  | maxdiff: 0.0
dhpreact        | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
dbngain         | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
dbnraw          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
dbnbias 